


DS Caveats: this code works turnkey for the given files at a specific point in time. There are several things that may change in future iterations, such as column names, data schemas and currency exchange rates. Before putting new data into this code, please review it section by section and make sure it conforms to your data's assumptions.

## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

## File Specifications

In [2]:
data_dir_june = '/Users/briandalessandro/Documents/CrossBoundary/E4I-Datasets/June_2019_DataShare/'
survey_data = '1_RF_Baseline_2018-07-24-FINAL.xlsx'
payment_data = 'CB_Innovation_Lab_Payment_Data_to_end_of_Jan_2019.csv'
consumption_data = 'CB_Innovation_Lab_Consumption_Data_to_end_of_Jan_2019.csv'
final_training_data = 'training_all_in.csv'

## Important Parameters

These should be reviewed before developing any model data

In [3]:
kenya2usd = 0.01 #exchange rate between Kenya and US Dollar
tanzania2usd = 0.00044 #exchange rate between tanzanie and US Dollar
tariff_start_date = '2016-10-15' #This is a subjective choice, to be made after exploring data
target_start_date = '2017-01-01'

## Workflow

In [4]:
survey_df = pd.read_excel(data_dir_june + survey_data)

#Clean column names
survey_df.columns = [f.replace(' ','_').replace('.','_').replace('__','_').lower() for f in survey_df.columns.values]

print('The survey data has {} (rows,columns)'.format(survey_df.shape))



The survey data has (2826, 570) (rows,columns)


### All Helper Functions

In [5]:


def split_mapper(row_vals, tag_dict):

    try:
        return ','.join([tag_dict.get(i) for i in row_vals.split(' ')])
    except:
        return 'none'
    

def process_cat_question(df, tag):
    tag_columns = [f.split('/')[1] for f in df.columns.values if tag in f and f != tag and '/' in f]
    tag_columns_drop = [f for f in df.columns.values if tag in f and f != tag]
    tag_dict = {str(i+1):str(k) for i,k in enumerate(tag_columns)}
    df[tag] = list(map(lambda x:split_mapper(x, tag_dict), df[tag]))
    df = df.drop(tag_columns_drop, axis=1)
    return df


def condense_multi_response_columns(df):
    '''
    Transform the expanded questions into a single field. This makes it easier to do more 
    general text analysis and feature exploration. This set of steps can be avoided altogether, 
    but it does clean up the column space nicely. 
    '''
    tags = set([c.split('/')[0] for c in df.columns.values if '/' in c])
    non_tags = set(['_47_rank_which_appli_ould_you_like_to_buy','meta','group_xq6xm27',
                    'group_hg4pl96','group_ss3zo89','rank_appliances_to_buy'])
    tags = list(tags - non_tags)

    for tag in tags:
        df = process_cat_question(df, tag)

    return df, tags



def remove_mostly_null_columns(df, missing_threshold=0.8):
    '''
    Drop any column that has an excess of missing values, determined by miss_thresh
    '''
    fields = df.columns.values
    missing_fields = []
    for f in fields:
        try:
            missing_pct = sum(np.isnan(df[f])) / float(df.shape[0])
            if missing_pct > missing_threshold:
                missing_fields.append(f)
        except:
            missing_pct = sum(df[f].isna()) / float(df.shape[0])
            if missing_pct > missing_threshold:
                missing_fields.append(f)
                
    df = df.drop(missing_fields, axis=1)
    return df

### Condense multi-answer columns into a single column

In [6]:
survey_df, multi_resp_tags = condense_multi_response_columns(survey_df)

print('The results has {} (rows,columns)'.format(survey_df.shape))


The results has (2826, 318) (rows,columns)


### Remove columns with mostly NULL values

In [7]:
survey_df = remove_mostly_null_columns(survey_df, missing_threshold=0.6)
print('The results has {} (rows,columns)'.format(survey_df.shape))


The results has (2826, 205) (rows,columns)


### Change Currency

In [9]:
#Most of this section is take from Michael's code
currency_cols = ['food','water','phone','transportation','health',
 'energy','education','debt','savings','rent','hh_expenses',
 'hh_income','purchase_of_energy','purchase_of_expensive_items','childrens_education']

for col in currency_cols:
    country_field = 'country'
    survey_df.loc[survey_df[country_field] == 'kenya', col] = survey_df[survey_df[country_field] == 'kenya'][col] * kenya2usd
    survey_df.loc[survey_df[country_field] == 'tanzania', col] = survey_df[survey_df[country_field] == 'tanzania'][col] * tanzania2usd

## Cap Outliers

In [10]:
#This is taken from Michael's code, which was copied from intern's code
# Age
survey_df.loc[survey_df.age < 18,'age']  = 18;

# Household_Sizes
survey_df.loc[survey_df.hh_size == 0, 'hh_size'] = 1;
survey_df.loc[survey_df.hh_size > 15, 'hh_size'] = 15;
survey_df.loc[survey_df.hh_size_over_60 > 15,'hh_size_over_60'] = 15;
survey_df.loc[survey_df.hh_size_between_5_18 > 15, 'hh_size_between_5_18'] =15;
survey_df.loc[survey_df.hh_size_under_5 > 15, 'hh_size_under_5'] = 15;

# Rooms
survey_df.loc[survey_df.nr_rooms > 10, 'nr_rooms'] = 10
survey_df.loc[survey_df.nr_rooms_used_for_sleeping > 10, 'nr_rooms_used_for_sleeping'] = 10
survey_df.loc[survey_df.nr_rooms == 0, 'nr_rooms'] = 1
survey_df.loc[survey_df.nr_rooms_used_for_sleeping == 0, 'nr_rooms_used_for_sleeping'] = 1

#Homework_time
# Baseline$Homework_time_hours[Baseline$Homework_time_hours > 16] <- 16
survey_df.loc[survey_df.homework_time_hours > 16, 'homework_time_hours'] = 16
#school_going_children 
survey_df.loc[survey_df.schoolgoing_children > 10, 'schoolgoing_children'] = 10
# Baseline$Schoolgoing_children[Baseline$Schoolgoing_children > 10] <- 10


# fetching times 
survey_df.loc[survey_df.fetching_time_minutes == 1200, 'fetching_time_minutes'] = 1200/10.
survey_df.loc[survey_df.fetching_times_per_day > 10, 'fetching_times_per_day'] = max(survey_df.fetching_times_per_day)/10.;

# Baseline$Fetching_times_per_day[Baseline$Fetching_times_per_day > 10] <- max(Baseline$Fetching_times_per_day)/10;


### Load and Process Payments Data

In [11]:
payments_df = pd.read_csv(data_dir_june + payment_data)
consumption_df = pd.read_csv(data_dir_june + consumption_data)

payments_df.columns = [f.replace(' ','_').replace('.','_').replace('__','_').lower() for f in payments_df.columns.values]
consumption_df.columns = [f.replace(' ','_').replace('.','_').replace('__','_').lower() for f in consumption_df.columns.values]


payments_df['month'] = map(lambda s: str(s)[0:7], payments_df.date)
consumption_df['month'] = map(lambda s: str(s)[0:7], consumption_df.date)

/Users/briandalessandro/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#Get Min/Max Tariffs over each month
payments_grp_m = payments_df[(payments_df.date>tariff_start_date)].groupby(['customer_id','month']).sum().reset_index()

consumption_grp_m = consumption_df[(consumption_df.date>tariff_start_date)].groupby(['customer_id','month']).sum().reset_index()

tariff_df = payments_grp_m[['customer_id','payment_local_curr','month']].merge(consumption_grp_m[['customer_id','consumption_kwh','month']],on=['customer_id','month'])

tariff_df['tariff'] = tariff_df['payment_local_curr'] / tariff_df['consumption_kwh'] 

tariff_df = tariff_df.merge(survey_df[['country','meter_number','developer_code','village_code']],left_on='customer_id',right_on='meter_number')
tariff_df.loc[tariff_df[country_field] == 'kenya', 'tariff'] = tariff_df[tariff_df[country_field] == 'kenya']['tariff'] * kenya2usd
tariff_df.loc[tariff_df[country_field] == 'tanzania', 'tariff'] = tariff_df[tariff_df[country_field] == 'tanzania']['tariff'] * tanzania2usd

clean_filt = (tariff_df['payment_local_curr']>1) & (tariff_df['consumption_kwh']>1)
tariff_min_max_df = tariff_df[clean_filt][['customer_id','tariff']].groupby('customer_id').agg([min,max]).reset_index()
tariff_min_max_df.columns = ['customer_id','tariff_min','tariff_max']

In [13]:
#Get avg tariff over all months
payments_grp = payments_df[(payments_df.date>tariff_start_date)].groupby(['customer_id']).sum().reset_index()
consumption_grp = consumption_df[(consumption_df.date>tariff_start_date)].groupby(['customer_id']).sum().reset_index()

tariff_df = payments_grp[['customer_id','payment_local_curr']].merge(consumption_grp[['customer_id','consumption_kwh']],on=['customer_id'])

tariff_df['tariff'] = tariff_df['payment_local_curr'] / tariff_df['consumption_kwh'] 

tariff_df = tariff_df.merge(survey_df[['country','meter_number','developer_code','village_code']],left_on='customer_id',right_on='meter_number')

tariff_df.loc[tariff_df[country_field] == 'kenya', 'tariff'] = tariff_df[tariff_df[country_field] == 'kenya']['tariff'] * kenya2usd
tariff_df.loc[tariff_df[country_field] == 'tanzania', 'tariff'] = tariff_df[tariff_df[country_field] == 'tanzania']['tariff'] * tanzania2usd



In [14]:
#Get tariffs by village / developer for imputation
keys=['developer_code','village_code']

payments_grp_d = payments_df[(payments_df.date>tariff_start_date)].groupby(keys).sum().reset_index()
consumption_grp_d = consumption_df[(consumption_df.date>tariff_start_date)].groupby(keys).sum().reset_index()

tariff_df_d = payments_grp_d[keys+ ['payment_local_curr']].merge(consumption_grp_d[keys+['consumption_kwh']],on=keys)



tariff_df_d['tariff_agg'] = tariff_df_d['payment_local_curr'] / tariff_df_d['consumption_kwh'] 


kenya = 1*(tariff_df_d.developer_code>3)
tariff_df_d['tariff_agg'] = tariff_df_d['tariff_agg']*((0.01)*kenya + (0.00044)*(1-kenya))
tariff_df_d.columns = [f.lower() for f in tariff_df_d.columns.values]

#Not sure about Dev_codes > 5, nonetheless, these aren't in survey data

In [15]:
#Now put it all together
tariff_df = tariff_df.merge(tariff_min_max_df,on='customer_id',how='left').merge(tariff_df_d[['developer_code','village_code','tariff_agg']], on=keys, how='left')


#Impute some of the crazy or missing values, using village aggregate
tariff_df.loc[tariff_df['consumption_kwh']<1, 'tariff'] = tariff_df[tariff_df['consumption_kwh']<1]['tariff_agg'] 
tariff_df.loc[tariff_df.tariff==np.Inf, 'tariff'] = tariff_df[tariff_df.tariff==np.Inf]['tariff_agg'] 
tariff_df.loc[np.isnan(tariff_df.tariff), 'tariff'] = tariff_df[np.isnan(tariff_df.tariff)]['tariff_agg'] 
tariff_df.loc[np.isnan(tariff_df.tariff_min), 'tariff_min'] = tariff_df[np.isnan(tariff_df.tariff_min)]['tariff_agg']
tariff_df.loc[np.isnan(tariff_df.tariff_max), 'tariff_max'] = tariff_df[np.isnan(tariff_df.tariff_max)]['tariff_agg'] 
tariff_df.loc[np.isnan(tariff_df.tariff), 'tariff']= tariff_df.tariff.median()

p99 = np.percentile(tariff_df.tariff,q=99)
tariff_df.loc[tariff_df.tariff>p99, 'tariff'] = p99

tariff_df.head()


,customer_id,payment_local_curr,consumption_kwh,tariff,country,meter_number,developer_code,village_code,tariff_min,tariff_max,tariff_agg
0,D8-80-39-76-3E-E4,2950.0,86.222,0.342140,kenya,D8-80-39-76-3E-E4,5,1,0.646959,0.646959,0.646959
1,D8-80-39-76-3E-F6,3300.0,32.809,1.005822,kenya,D8-80-39-76-3E-F6,5,1,0.646959,0.646959,0.646959
2,D8-80-39-76-3E-F8,16800.0,279.576,0.600910,kenya,D8-80-39-76-3E-F8,5,1,0.646959,0.646959,0.646959
3,D8-80-39-76-3F-04,17600.0,352.804,0.498861,kenya,D8-80-39-76-3F-04,5,1,0.646959,0.646959,0.646959
4,D8-80-39-76-3F-06,4500.0,74.544,0.603670,kenya,D8-80-39-76-3F-06,5,1,0.646959,0.646959,0.646959


### Now Build Target Data

Use 3-12 months for each individual, with outliers removed

In [16]:

cons_df = pd.read_csv(data_dir_june + consumption_data)

cons_df.columns = [f.replace(' ','_').replace('.','_').replace('__','_').lower() for f in cons_df.columns.values]


keep_fields = ['date','customer_id','consumption_kwh']

group1_df = cons_df[keep_fields].groupby(['date','customer_id']).agg([sum]).reset_index()
group1_df.columns = ['date','customer_id','consumption_kwh']
group1_df['date'] = pd.to_datetime(group1_df['date'])

#Get first day for each customer and compute tenure for each entry
first_day_df = group1_df[['customer_id','date']].groupby(['customer_id']).min().reset_index()
first_day_df.rename({'date':'first_date'}, axis=1, inplace=True)
group1_df = group1_df.merge(first_day_df, on='customer_id')
group1_df['tenure'] = (group1_df['date'] - group1_df['first_date']).dt.days

#We don't trust anything before 2017
group1_df = group1_df[(group1_df.date>=target_start_date)]

#Get all billings between first 90 days and 1 year
group1_df = group1_df[(group1_df.tenure>90) & (group1_df.tenure<=365)]

#Get min/max billing dates within this range
group1_min_max = group1_df[['customer_id','date']].groupby(['customer_id']).agg([min,max]).reset_index()
group1_min_max.columns = ['customer_id', 'start_date','end_date']

#Get total usage
group1_agg = group1_df[['customer_id','consumption_kwh']].groupby(['customer_id']).sum().reset_index()
group1_agg = group1_agg.merge(group1_min_max, on='customer_id')
group1_agg['days'] = (group1_agg['end_date'] - group1_agg['start_date']).dt.days + 1
group1_agg['avg_consumption'] = group1_agg['consumption_kwh'] / group1_agg['days']

#Final Target Data
target_df = group1_agg[['customer_id','start_date','end_date','avg_consumption']]


/Users/briandalessandro/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Join Together All Features

In [17]:
tariff_df.columns = [f.replace(' ','_').replace('.','_').lower() for f in tariff_df.columns.values]
target_df.columns = [f.replace(' ','_').replace('.','_').lower() for f in target_df.columns.values]


model_df = survey_df.merge(tariff_df[['customer_id','tariff','tariff_min','tariff_max']], left_on='meter_number',right_on='customer_id')

model_df = model_df.merge(target_df, on='customer_id')

#Clean features one more time, adding in the '/' as well
model_df.columns = [f.replace('/','_').replace(' ','_').replace('.','_').lower() for f in model_df.columns.values]

print('The model data has {} (rows,columns)'.format(model_df.shape))


The model data has (1859, 212) (rows,columns)


<b>Developer note:</b> I took everything up until this point and fed it into the SparkBeyond platform. The goal of which was to identify if any interesting features could be discovered. In particular, looking for interesting transformations of text or geo-stamped fields. The section below is the set of features that were found to be predictive. But everything above can constitue 'raw' inputs, upon which future feature engineering can be done.

## Features Added after Automatic Feature Engineering

In [18]:


def contain_in_list_values(column, value):
    return [int(value in f.lower()) for f in column]

drop_list = []

#Straightforward ones
model_df['has_decoder_energy_source'] = 1*(model_df.decoder_energy_source != "none") 
model_df['decoder_energy_source_is_electricity'] = 1*(model_df.decoder_energy_source == "Electricity") 


model_df['energy_source_for_cooking_is_coal'] = 1*(model_df.energy_source_for_cooking == "coal") #Note - just coal, worth an investigation
model_df['energy_source_for_cooking_not_none'] = 1*(model_df.energy_source_for_cooking != "none") #Note - just coal, worth an investigation


model_df['hh_source_of_income_is_corn'] = 1*(model_df.hh_source_of_income == "corn") 
model_df['hh_source_of_income_has_commerce'] = contain_in_list_values(model_df.hh_source_of_income,"commerce")


model_df['energy_source_for_lighting_is_electricity'] = 1*(model_df.energy_source_for_lighting == "Electricity") 
model_df['energy_source_for_lighting_has_solar'] = contain_in_list_values(model_df.energy_source_for_lighting,"solar") #this amounts to higher electricity usage interestingly enough
model_df['energy_source_for_lighting_is_none'] = 1*(model_df.energy_source_for_lighting == "none") 


model_df['transport_use_or_ownership_has_motorbike'] = contain_in_list_values(model_df.transport_use_or_ownership, "motorbike")
model_df['transport_use_or_ownership_has_car'] = contain_in_list_values(model_df.transport_use_or_ownership, "car")

model_df['group_involvement_not_none'] = 1*(model_df.group_involvement != "none")

model_df['uses_of_non_self_generated_electricity_has_fan'] = contain_in_list_values(model_df.uses_of_non_self_generated_electricity, "fan")
model_df['uses_of_non_self_generated_electricity_has_phone_charging'] = contain_in_list_values(model_df.uses_of_non_self_generated_electricity, "phone_charging")
model_df['uses_of_non_self_generated_electricity_count'] = [len(f.split(','))*(f!='none') for f in model_df.uses_of_non_self_generated_electricity]
model_df['uses_of_non_self_generated_electricity_has_radio'] = contain_in_list_values(model_df.uses_of_non_self_generated_electricity, "radio")


model_df['occupation_not_none'] = 1*(model_df.occupation != "none")

model_df['transport_use_or_ownership_has_car'] = contain_in_list_values(model_df.transport_use_or_ownership, "car")

model_df['income_generating_activity_has_salary_work'] = contain_in_list_values(model_df.income_generating_activity, "salary_work")


model_df['how_often_in_the_e_following_feelings_is_ok'] = 1*(model_df.how_often_in_the_e_following_feelings == 'OK')




In [19]:
#These vars when present have the value OK - convert these to binary
cat_vars = ['how_often_in_the_e_following_feelings',
 'group_hg4pl96_group_xl3ae56__20_how_many_of_each_s_your_household_own',
 'group_hg4pl96_group_ti5qr72__25_who_makes_the_fo_ion_in_the_household',
 'agree_or_disagree_wi_following_statements',
 'group_xq6xm27_group_cz2uj23_group_wh9vo30__35_how_many_of_each_light_your_household',
 'group_hg4pl96_group_cr5xn88__24_how_much_does_yo_ing_local_currency',
 'group_hg4pl96_group_ob7yj27_the_households_total_per_month_on_average']


for v in cat_vars:
    model_df[v] = 1*(model_df[v]=='OK')

    
#Simple transformations
def f(x, offset=0): 
    try:
        slp = int(x[0:2]) + int(x[3:5]) / 60.0
        if slp < 8:
            slp += offset
        return slp   
    except:
        return np.NaN


model_df['sleep_time'] = list(map(lambda x: f(x,24), model_df['sleep_time'].values))
model_df['wake_up_time'] = list(map(lambda x: f(x), model_df['wake_up_time'].values))


model_df = model_df.drop(multi_resp_tags, axis=1)

print('The model data has {} (rows,columns)'.format(model_df.shape))


The model data has (1859, 192) (rows,columns)


## Missing Value Imputation

In [20]:
#These we just fill missing values with 0
fill_zero_list = ['good','bad','afraid','angry']

for v in fill_zero_list:
    model_df[v] = model_df[v].fillna(value=0)

In [21]:
#These are features that won't go into a model, so don't need imputation
non_model_features = ['start_date','submission_time','end_date','device_id','country','developer_code', 
                      'village_code', 'joint_code', 'gps_coordinates',
                       'gps_coordinates_latitude', 'gps_coordinates_longitude',
                       'gps_coordinates_altitude', 'gps_coordinates_precision',
                       'meter_number','survey_version','meta_instanceid', 'id', 'uuid', 'index', 
                       'parent_index']

fill_columns = list(set(model_df.columns.values) - set(non_model_features))


def get_missval_signal(df, var):
    '''
    get a t-stat on the difference in the target variable for missing vs non missing
    '''
    df2 = pd.DataFrame({'x':df[var].isna(), 'y':df.avg_consumption})
    dfg = df2.groupby('x').agg([len,np.mean,np.std]).reset_index()
    dfg.columns = ['x', 'n','mu','sig']
    dfg_f = dfg[(dfg.x==False)]
    dfg_t = dfg[(dfg.x==True)]
    return (dfg_t.mu.values - dfg_f.mu.values) / np.sqrt(dfg_t.sig.values**2 / dfg_t.n.values + dfg_f.sig.values**2 / dfg_f.n.values)


mv_has_signal = []
mv_str_type = []
for var in fill_columns:
    if np.abs(get_missval_signal(model_df, var)) > 2.2: #roughly 97.5% two tailed statistical significance
        mv_has_signal.append(var)
        model_df[var + '_is_missing'] = 1*model_df[var].isna()
        
    is_str = type(model_df[var][1-(model_df[var].isna())].values[0]) in [str]
        
    #Convert numerics to median value
    if not is_str:
        model_df[var] =  model_df[var].fillna(model_df[var].median(skipna=True))
        
    else:
        mv_str_type.append(var)


/Users/briandalessandro/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


### Final Output

In [22]:
model_df.to_csv(data_dir_june + final_training_data, index=False)
